In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from boundary import *
import rootfinding as rf
custom_rf = {'rootfind_open': rf.newton, 'rootfind_bracketing': rf.bisect}

In [2]:
def billiard_propagator_gen(r, v, bdy):
    s = bdy.linear_intersect_cart(r, v)
    if np.any(np.sign(bdy.coords_cart(s) - r) != np.sign(v)):
        # Then we found the intercept in the wrong direction! Try again.
        s = bdy.linear_intersect_param(s, v)  # Excludes passed root s
    while True:
        tangent = bdy.tangent_cart(s)
        v_tan = np.dot(v, tangent)
        yield s, v_tan
        v = 2 * v_tan * tangent - v
        s = bdy.linear_intersect_param(s, v)


def init_state(r, v, boundary):
    state = {'s_list': [],
             'v_tan_list': [],
             'trajectory': np.c_[r],
             'propagator': billiard_propagator_gen(r, v, bdy)}
    return state


def propagate(state, bounces):
    new_s, new_v_tan = zip(*(state['propagator'].next()
                             for _ in xrange(bounces)))
    state['s_list'].extend(new_s)
    state['v_tan_list'].extend(new_v_tan)
    state['trajectory'] = np.c_[state['trajectory'],
                                bdy.coords_cart(np.array(new_s))]

In [12]:
%matplotlib auto
r0 = np.array([0.6, 0.2])
theta = 0.1
v0 = np.array([np.cos(theta), np.sin(theta)])
#bdy = UnitCircleBoundary(**custom_rf)
bdy = BeanBoundary(0.16, 0.1, 2.0, 1.0)#**custom_rf)

default_bounces = 50
max_bounces = 500
default_deltay = 0
max_deltay = 1000

fig, axlist = plt.subplots(1, 2)
bdyline = bdy.coords_cart(np.arange(0, 2*np.pi, 0.01))
axlist[0].plot(bdyline[0], bdyline[1])
# Plot trajectory line object; I'll send data to it later 
traj_line, = axlist[0].plot([0, 0], [0, 0], label='Trajectory')
poinc_ax = axlist[1]
plt.axis('equal')

# Make Sliders (with their own axis artists) for interactive control.
axcolor = 'lightgoldenrodyellow'
axbounces = plt.axes([0.2, 0.03, 0.65, 0.02])#, facecolor=axcolor)
sbounces = Slider(axbounces, 'Number of bounces',
                  1, max_bounces, valinit=default_bounces)
axdeltay = plt.axes([0.2, 0.005, 0.65, 0.02])#, facecolor=axcolor)
sdeltay = Slider(axdeltay, r'$\Delta y_0$ / $(y_0 * 10^{-8})$',
                 -max_deltay, max_deltay, valinit=default_deltay)


def propagate_plots(state, n, traj_line, poinc_ax):
    missing = n+1 - state['trajectory'].shape[1]
    poinc_ax.clear()
    if missing <= 0:
        traj_line.set_xdata(state['trajectory'][0, 0:n+1])
        traj_line.set_ydata(state['trajectory'][1, 0:n+1])
        poinc_ax.scatter(state['s_list'][0:n+1],
                         state['v_tan_list'][0:n+1])
    else:
        propagate(state, missing)    
        traj_line.set_xdata(state['trajectory'][0])
        traj_line.set_ydata(state['trajectory'][1])
        poinc_ax.scatter(state['s_list'],
                         state['v_tan_list'])
    fig.canvas.draw()


def reset_plots(r, v, boundary, n, traj_line, poinc_ax):
    state = init_state(r, v, boundary)
    propagate_plots(state, n, traj_line, poinc_ax)
    return state

# Initialize plotline data for default Slider settings.
state = reset_plots(r0 * np.array([1, 1 + sdeltay.val*1e-8]), v0, bdy,
                    int(sbounces.val), traj_line, poinc_ax)
# Configure our Sliders to update the plotlines.
# The Sliders will pass their current value to the on_changed event callback
sbounces.on_changed(lambda val: propagate_plots(state, int(val),
                                                traj_line, poinc_ax))
sdeltay.on_changed(lambda val: reset_plots(r0 * np.array([1, 1 + val*1e-8]),
                                           v0, bdy, int(sbounces.val),
                                           traj_line, poinc_ax))

# plt.show()

Using matplotlib backend: Qt5Agg


0

fig, ax = plt.subplots(1,1)
print len(state['s_list'])
ax.scatter(state['s_list'], state['v_tan_list'])
plt.show()